In [2]:
from utils.df_handle import *

In [3]:
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y%m%d")

In [4]:
datenow_mns1

'2022-03-28'

In [5]:
sql = \
f"""
WITH T1 as
(
SELECT
BranchID,
SlsperID,
Lat,
Lng,
CustID,
Type as Typ,
NumberCICO,
UpdateTime as CheckInUpdateTime
from dbo.AR_SalespersonLocationTrace 
where CAST (UpdateTime as DATE) >= '{datenow_mns1}'
and CAST (UpdateTime as DATE) <= '{datenow_mns1}'
and Type = 'IO'
--ORDER BY UpdateTime ASC
)

--select top 100 SUBSTRING(Type,1,2), Type from dbo.AR_SalespersonLocationTrace where CAST (UpdateTime as DATE) = '2022-03-25'
, T2 as
(
SELECT
BranchID,
SlsperID,
SUBSTRING(Type,3,20) as DeOrderNbr,
UpdateTime as DE_UpdateTime,
NumberCICO
from
dbo.AR_SalespersonLocationTrace
where CAST (UpdateTime as DATE) >= '{datenow_mns1}'
and CAST (UpdateTime as DATE) <= '{datenow_mns1}'
and SUBSTRING(Type,1,2) = 'DE'
)

Select
T1.BranchID,
T1.SlsperID,
Lat,
Lng,
CustID,
Typ,
T1.NumberCICO,
CheckInUpdateTime,
DeOrderNbr,
DE_UpdateTime
from T1
LEFT JOIN T2 ON
T1.NumberCICO = T2.NumberCICO
and T1.BranchID = T2.BranchID
and T1.SlsperID = T1.SlsperID
ORDER BY CheckInUpdateTime ASC
"""

In [12]:
TRACE = get_ms_df(sql)

In [5]:
conn = get_conn()

In [7]:
TRACE.columns = lower_col(TRACE)

In [8]:
TRACE.to_sql(name="d_trace_location", con=conn, index=False, if_exists='replace')

In [14]:
TRACE.DeOrderNbr = np.where(TRACE.DeOrderNbr.isna(), None, TRACE.DeOrderNbr)
# TRACE.DE_UpdateTime = np.where(TRACE.DE_UpdateTime.isna(), None, TRACE.DE_UpdateTime)

In [16]:
TRACE.DE_UpdateTime = np.where(TRACE.DE_UpdateTime.isna(), None, TRACE.DE_UpdateTime)

In [19]:
TRACE.DE_UpdateTime = pd.to_datetime(TRACE.DE_UpdateTime, dayfirst=True)

In [22]:
TRACE.DE_UpdateTime.fillna(datetime(1900,1,1), inplace=True)

In [34]:
execute_values_insert(TRACE, "d_trace_location")

Data inserted using execute_values() successfully..


In [31]:
TRACE.columns

Index(['BranchID', 'SlsperID', 'Lat', 'Lng', 'CustID', 'Typ', 'NumberCICO',
       'CheckInUpdateTime', 'DeOrderNbr', 'DE_UpdateTime'],
      dtype='object')

In [33]:
max(TRACE.CustID.str.len())

13